Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline

Loading the dataset

In [ ]:
df = pd.read_csv('/content/data - Copy - Copy - Copy - Copy - Copy.csv')
df = df.drop('Humidity (%)', axis=1)

Feature set and target variable

In [ ]:
X = df.drop('SOC (%)', axis=1)
y = df['SOC (%)']

Scale features with MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

Applying Polynomial Features on the scaled data

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_scaled)

Train-test split

In [ ]:
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

Defining models

In [ ]:
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
}

Hyperparameter tuning using GridSearchCV

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}

Performing grid search for each model

In [ ]:
best_models = {}
for name, model in models.items():
    print(f"Hyperparameter tuning for {name}...")
    if name == 'RandomForest':
        grid_search = GridSearchCV(model, param_grid_rf, cv=5, scoring='neg_mean_absolute_error')
    elif name == 'GradientBoosting':
        grid_search = GridSearchCV(model, param_grid_gb, cv=5, scoring='neg_mean_absolute_error')

    grid_search.fit(X_train_poly, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Best hyperparameters for {name}: {grid_search.best_params_}")

Hyperparameter tuning for RandomForest...
Best hyperparameters for RandomForest: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Hyperparameter tuning for GradientBoosting...
Best hyperparameters for GradientBoosting: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


Evaluating models and making predictions for new scenarios

In [ ]:
for name, model in best_models.items():
    y_pred = model.predict(X_test_poly)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"\n{name} Model Mean Absolute Error: {mae:.2f}%")

    # new_scenarios = [
    #     {'Voltage (V)': 3.5, 'Current (A)': 2.5, 'Temperature (°C)': 25.0},
    #     {'Voltage (V)': 4.2, 'Current (A)': -1.2, 'Temperature (°C)': 30.0},
    #     {'Voltage (V)': 3.8, 'Current (A)': 3.1, 'Temperature (°C)': 27.0},
    #     {'Voltage (V)': 4.0, 'Current (A)': -0.8, 'Temperature (°C)': 29.0},
    #     {'Voltage (V)': 3.9, 'Current (A)': 2.0, 'Temperature (°C)': 26.0}
    # ]

    # new_scenarios = [
    # {'Voltage (V)': 3.6, 'Current (A)': 1.8, 'Temperature (°C)': 24.0},
    # {'Voltage (V)': 4.1, 'Current (A)': -1.5, 'Temperature (°C)': 29.5},
    # {'Voltage (V)': 3.7, 'Current (A)': 2.8, 'Temperature (°C)': 28.0},
    # {'Voltage (V)': 4.0, 'Current (A)': -1.0, 'Temperature (°C)': 27.5},
    # {'Voltage (V)': 3.9, 'Current (A)': 2.3, 'Temperature (°C)': 25.5}
    #  ]

    new_scenarios = [
    {'Voltage (V)': 48.0, 'Current (A)': 7.49, 'Temperature (°C)': 45.23},
    {'Voltage (V)': 44.0, 'Current (A)': -1.5, 'Temperature (°C)': 29.5},
    {'Voltage (V)': 47.0, 'Current (A)': 2.8, 'Temperature (°C)': 28.0},
    {'Voltage (V)': 4.0, 'Current (A)': -1.0, 'Temperature (°C)': 27.5},
    {'Voltage (V)': 3.9, 'Current (A)': 2.3, 'Temperature (°C)': 25.5}
      ]

    # 'Voltage (V)': [41.0, 44.0, 47.0, 48.0],
    # 'Current (A)': [5.0, 10.0, 15.0, 19.0],
    # 'Temperature (°C)': [20, 25, 30, 29],
    # 'Humidity (%)': [45, 50, 55, 59]

    new_data = pd.DataFrame(new_scenarios)
    new_data_scaled = scaler.transform(new_data)

    new_data_poly = poly.transform(new_data_scaled)

    new_soc_predictions = model.predict(new_data_poly)
    new_soc_predictions = np.clip(new_soc_predictions, 0, 100)

    for i, soc in enumerate(new_soc_predictions):
        print(f"Scenario {i + 1}: {new_scenarios[i]} -> Predicted SoC ({name}): {soc:.2f}%")



RandomForest Model Mean Absolute Error: 0.09%
Scenario 1: {'Voltage (V)': 48.0, 'Current (A)': 7.49, 'Temperature (°C)': 45.23} -> Predicted SoC (RandomForest): 99.76%
Scenario 2: {'Voltage (V)': 44.0, 'Current (A)': -1.5, 'Temperature (°C)': 29.5} -> Predicted SoC (RandomForest): 49.86%
Scenario 3: {'Voltage (V)': 47.0, 'Current (A)': 2.8, 'Temperature (°C)': 28.0} -> Predicted SoC (RandomForest): 87.52%
Scenario 4: {'Voltage (V)': 4.0, 'Current (A)': -1.0, 'Temperature (°C)': 27.5} -> Predicted SoC (RandomForest): 51.80%
Scenario 5: {'Voltage (V)': 3.9, 'Current (A)': 2.3, 'Temperature (°C)': 25.5} -> Predicted SoC (RandomForest): 51.79%

GradientBoosting Model Mean Absolute Error: 0.12%
Scenario 1: {'Voltage (V)': 48.0, 'Current (A)': 7.49, 'Temperature (°C)': 45.23} -> Predicted SoC (GradientBoosting): 99.97%
Scenario 2: {'Voltage (V)': 44.0, 'Current (A)': -1.5, 'Temperature (°C)': 29.5} -> Predicted SoC (GradientBoosting): 50.07%
Scenario 3: {'Voltage (V)': 47.0, 'Current (A)': 

Polynomial Model (manual implementation, using polynomial features only)

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(degree=2), RandomForestRegressor(random_state=42))
poly_model.fit(X_train_poly, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

Evaluating polynomial model performance

In [ ]:
y_pred_poly = poly_model.predict(X_test_poly)
mae_poly = mean_absolute_error(y_test, y_pred_poly)
print(f"Polynomial Model Mean Absolute Error: {mae_poly:.2f}%")

new_soc_predictions_poly = poly_model.predict(new_data_poly)
new_soc_predictions_poly = np.clip(new_soc_predictions_poly, 0, 100)

for i, soc in enumerate(new_soc_predictions_poly):
    print(f"Scenario {i + 1}: {new_scenarios[i]} -> Predicted SoC (Poly): {soc:.2f}%")

Polynomial Model Mean Absolute Error: 0.10%
Scenario 1: {'Voltage (V)': 48.0, 'Current (A)': 7.49, 'Temperature (°C)': 45.23} -> Predicted SoC (Poly): 99.73%
Scenario 2: {'Voltage (V)': 44.0, 'Current (A)': -1.5, 'Temperature (°C)': 29.5} -> Predicted SoC (Poly): 49.82%
Scenario 3: {'Voltage (V)': 47.0, 'Current (A)': 2.8, 'Temperature (°C)': 28.0} -> Predicted SoC (Poly): 87.53%
Scenario 4: {'Voltage (V)': 4.0, 'Current (A)': -1.0, 'Temperature (°C)': 27.5} -> Predicted SoC (Poly): 58.51%
Scenario 5: {'Voltage (V)': 3.9, 'Current (A)': 2.3, 'Temperature (°C)': 25.5} -> Predicted SoC (Poly): 58.52%
